In [51]:
import numpy as np
import csv
import re

import pandas as pd
import pdfplumber


In [52]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN


def check(strr):
    my_re = re.compile(r'[A-Za-z]', re.S)
    res = re.findall(my_re, strr)
    if len(res):
        return True
    else:
        return False


def dataset(filepath):
    nname = []
    iinformation = []
    with open(filepath, 'r+') as f:
        readers = csv.reader(f, delimiter=",")
        x = list(readers)[1:]
        data = np.array(x)
        for line in data:
            nname.append(line[0])
            iinformation.append([np.double(line[i]) for i in range(1, len(line))])
        return nname, iinformation


def test_one(file_path):
    pdf = pdfplumber.open(file_path)
    page = pdf.pages[0]
    word = page.extract_words(y_tolerance=-1)

    with open("output_csv/" + pdf.metadata['Title'] + ".csv", 'w', newline='') as f:
        row = list(word[0].keys())[0:6]
        row.pop(3)
        write = csv.writer(f)
        write.writerow(row)
        for wword in word:
            if check(wword['text']):
                continue
            # row = list(wword.values())[0:6]
            # write = csv.writer(f)
            # write.writerow(row)
            elif wword['text'].__contains__('.'):
                row = list(wword.values())[0:6]
                row.pop(3)
                write = csv.writer(f)
                write.writerow(row)

    name, information = dataset("output_csv/" + pdf.metadata['Title'] + ".csv")

    X = StandardScaler().fit_transform(information)
    db = DBSCAN(eps=0.3,min_samples=3).fit(X)
    # print(name)
    # print(db.labels_)
    # print(db.core_sample_indices_)

    date = pd.read_csv("output_csv/" + pdf.metadata['Title'] + ".csv")
    # print(date)
    # print(db.labels_)
    date['label'] = db.labels_
    date.to_csv("output_csv/" + pdf.metadata['Title'] + ".csv",index=False)
    # print(pd.read_csv("output_csv/" + pdf.metadata['Title'] + ".csv"))

def separate(date):
    for lable in date[0]:
        if lable != -1:


In [53]:

path = "data_pdf/cp_8_6.pdf"
test_one(path)

    text        x0        x1    doctop    bottom
0   0.55  146.7060  158.3820   16.9175   22.9175
1   2.10   82.3721   93.8321   63.7092   69.7092
2   0.30  168.3882  180.0642   64.8703   70.8703
3   2.00   82.3721   93.8321   71.7072   77.7072
4   0.25  168.3882  180.0642   72.8683   78.8683
5   0.50  241.8613  253.5373   74.1301   80.1301
6   1.90   82.3721   93.8321   79.7052   85.7052
7   0.18  168.3882  180.0642   80.8663   86.8663
8   0.65  155.4072  167.0832  124.1100  130.1100
9   0.60  155.4072  167.0832  132.1080  138.1080
10  0.55  155.4072  167.0832  140.1060  146.1060
11  0.60   29.3760   41.0520  180.6130  186.6130
12  0.05  144.7666  156.4426  186.5969  192.5969
13  0.55   29.3760   41.0520  188.6110  194.6110
14  0.02  144.7666  156.4426  194.5949  200.5949
15  0.50   29.3760   41.0520  196.6090  202.6090
16  0.20  137.2397  148.9157  217.3801  223.3801
[-1  0  1  0  1 -1  0  1  2  2  2  3 -1  3 -1  3 -1]
    text        x0        x1    doctop    bottom  label
0   0.55 